# Ensemble - CBAM

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow_addons as tfa

from glob import glob
from tqdm import tqdm

import cv2

from imgaug import augmenters as iaa
import imgaug as ia

import argparse
import wandb
from wandb.keras import WandbCallback
wandb.init(project="DACON_235896", name="Ensemble")

parser = argparse.ArgumentParser(description='Ensemble')
parser.add_argument('--resize_size', default=224, type=int)
parser.add_argument('--randaugment_n', default=3, type=int)
parser.add_argument('--randaugment_m', default=7, type=int)
parser.add_argument('--optimizer', default="sgd", type=str) # sgd or adam
parser.add_argument('--learning_rate', default=0.01, type=float)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--epochs', default=200, type=int)
parser.add_argument('--n_splits', default=10, type=int)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

resize_size=args.resize_size
BATCH_SIZE=args.batch_size
EPOCHS=args.epochs
N_SPLITS=args.n_splits
SEED=args.seed

def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    ia.seed(seed)

set_seeds()

## Preprocessing

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (resize_size, resize_size))
    return img

X = np.array([img_load(i) for i in tqdm(glob('data/train/*.png'))])
X_test = np.array([img_load(i) for i in tqdm(glob('data/test/*.png'))])

y = pd.read_csv("data/train.csv")["label"]
y_encoder = {key : value for key, value in zip(np.unique(y), range(len(np.unique(y))))}
y = np.array([y_encoder[k] for k in y])

X.shape, X_test.shape, y.shape, X.dtype

In [ ]:
pd.read_csv("data/train.csv")["label"].value_counts()

In [ ]:
for k, v in y_encoder.items():
    plt.figure(figsize=(21,20))
    for i in range(10):
        plt.subplot(1, 11, i+1)
        plt.imshow(X[y==v][i])
        plt.title(k)
        plt.axis("off")
    plt.show()

## Modelling

### Attention Modules

In [ ]:
def se_block(input_feature, ratio=8):
    
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]
    
    se_feature = layers.GlobalAveragePooling2D()(input_feature)
    se_feature = layers.Reshape((1, 1, channel))(se_feature)
    se_feature = layers.Dense(channel // ratio,
                              activation='relu',
                              kernel_initializer='he_normal',
                              use_bias=True,
                              bias_initializer='zeros')(se_feature)
    se_feature = layers.Dense(channel,
                              activation='sigmoid',
                              kernel_initializer='he_normal',
                              use_bias=True,
                              bias_initializer='zeros')(se_feature)
    
    if K.image_data_format() == 'channels_first':
        se_feature = layers.Permute((3, 1, 2))(se_feature)
        
    se_feature = layers.multiply([input_feature, se_feature])
    
    return se_feature


def cbam_block(cbam_feature, ratio=8):
    
    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    
    return cbam_feature


def channel_attention(input_feature, ratio=8):
    
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]
    
    shared_layer_one = layers.Dense(channel//ratio,
                                    activation='relu',
                                    kernel_initializer='he_normal',
                                    use_bias=True,
                                    bias_initializer='zeros')
    shared_layer_two = layers.Dense(channel,
                                    kernel_initializer='he_normal',
                                    use_bias=True,
                                    bias_initializer='zeros')

    avg_pool = layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = layers.Reshape((1,1,channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)
    
    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = layers.Reshape((1,1,channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)
    
    cbam_feature = layers.Add()([avg_pool,max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)
    
    if K.image_data_format() == "channels_first":
        cbam_feature = layers.Permute((3, 1, 2))(cbam_feature)
        
    return layers.multiply([input_feature, cbam_feature])


def spatial_attention(input_feature):
    
    kernel_size = 7
    
    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = layers.Permute((2,3,1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature
        
    avg_pool = layers.Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    max_pool = layers.Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    concat = layers.Concatenate(axis=3)([avg_pool, max_pool])
    cbam_feature = layers.Conv2D(filters = 1,
                                 kernel_size=kernel_size,
                                 strides=1,
                                 padding='same',
                                 activation='sigmoid',
                                 kernel_initializer='he_normal',
                                 use_bias=False)(concat)
    
    if K.image_data_format() == "channels_first":
        cbam_feature = layers.Permute((3, 1, 2))(cbam_feature)
        
    return layers.multiply([input_feature, cbam_feature])

### Model

In [ ]:
def cbam(activation='relu'):

    inp = tf.keras.Input(shape=(resize_size, resize_size, 3))
    x = layers.Rescaling(1./255)(inp)

    x = layers.Conv2D(32, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(32, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = cbam_block(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(64, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = cbam_block(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(128, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = cbam_block(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(256, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(256, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = cbam_block(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(512, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(512, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = cbam_block(x)

    x = layers.Conv2D(1024, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(1024, 3, 1, "same", kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = cbam_block(x)

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(256, activation=activation)(x)
    x = layers.Dense(64, activation=activation)(x)
    oup = layers.Dense(len(np.unique(y)), activation="softmax")(x)
    model = tf.keras.Model(inputs=inp, outputs=oup)
    
    return model

### Augmentation

In [ ]:
rand_aug = iaa.RandAugment(n=args.randaugment_n, m=args.randaugment_m)

def augment(images):
    images = tf.cast(images, tf.uint8)
    return rand_aug(images=images.numpy())

In [ ]:
def tta(model, imgs):
    tta_pred=[]
    tta_pred.append(model.predict(imgs))
    tta_pred.append(model.predict(imgs[:,:,::-1]))
    return np.mean(tta_pred, axis=0)

## Training

In [ ]:
idx=0
pred_ensemble=[]

skf = StratifiedKFold(n_splits=N_SPLITS)
for train_index, val_index in skf.split(X, y):
    
    idx+=1
    
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .shuffle(len(X_train))
        .batch(BATCH_SIZE)
        .map(lambda x, y: (tf.py_function(augment, [x], [tf.float32])[0], y),
             num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    model = cbam()
    
    lr = tf.keras.optimizers.schedules.CosineDecay(args.learning_rate, decay_steps=1000)
    if args.optimizer == "sgd":
        optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
    elif args.optimizer == "adam":
        optim = tf.keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(
        optimizer=optim,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    checkpoint_filepath=f"load_model/{parser.description}_{idx}"

    checkpoint_callback = [
        tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True
        )
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[checkpoint_callback, WandbCallback()],
    )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss=history.history['loss']
    val_loss=history.history['val_loss']

    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.show()

    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
    
    model.load_weights(checkpoint_filepath)
    
    pred_prob = tta(model, X_test)
    pred_ensemble.append(pred_prob)

## Inference

In [ ]:
pred_prob = np.mean(pred_ensemble, axis=0)
pred = np.argmax(pred_prob, axis=1)
y_decoder = {value : key for key, value in y_encoder.items()}
pred = np.array([y_decoder[v] for v in pred])

pd.Series(pred).value_counts()

In [ ]:
n=pd.Series(pred).value_counts().min()

for k, v in y_decoder.items():
    plt.figure(figsize=(21,20))
    for i in range(n):
        plt.subplot(1, n+1, i+1)
        plt.imshow(X_test[pred==v][i])
        plt.title(v)
        plt.axis("off")
    plt.show()

In [ ]:
submission = pd.read_csv("data/sample_submission.csv")
submission["label"] = pred
submission.to_csv(f"{parser.description}.csv", index=False)